# MATHS1004 Mathematics for Data Science I
## Computer Lab 4

This lab is all about applications of eigenvalues and eigenvectors. We'll start with an example (taken from lectures in week 6) of eigenvalues, which we learn the theory of in week 7, and then a more data-driven example demonstrating the application of eigenvectors to principal component analysis (PCA), which we'll cover the theory for by week 8.

### Eigenvalues application: the Northern Spotted Owl

As discussed towards the end of the Thursday week 6 lecture (5 September), the growth in population numbers of juvenile and adult spotted owls can be modelled by the system of linear equations
$$
\begin{align*}
j_{k+1} &=  a_k/3\\
a_{k+1} & = \beta j_k + 0.9a_k
\end{align*}
$$
where
$j_k$ and $a_k$ represent the numbers of (female) juvenile and adult owls respectively in year $k$,
and $\beta$ is the proportion of juvenile owls who survive to adulthood after year $k$.
the 1/3 is the proportion of juveniles that are born to the number of adults in year $k$,
and the 0.9 is the proportion of adults who survive year $k$.


We can write this in the following matrix form:
$$
\mathbf{x}_{k+1} = A\mathbf{x}_k
$$
where
$$
\mathbf{x}_k = 
\begin{bmatrix}
j_k\\
a_k
\end{bmatrix}
$$
and
$$
A =
\begin{bmatrix}
0 & 1/3\\
\beta & 0.9
\end{bmatrix}
$$
(check that you believe this.)

Code this system up in the cells below using `numpy` for the matrices, and use a `for` loop to simulate this population for 100 years (say), starting from some initial population vector `x` (make up some numbers), and some value of $\beta$ (try $\beta = 0.1$ to start with like I did in the lecture if you want). Make a plot of the population sizes over time if you like, or just compare the population size after 100 years to the initial population size.


[Note: the past 3 labs have given you all the tools you need to be able to do this! You shouldn't feel afraid to give it a try. But you can also just go back and pause the 5 September lecture video at the appropriate point to copy down the code.]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


Once you have the code: do some exploration! Try changing:
- The initial population size `x`;
- The number of years we are simulating the population for (`N_steps` in the lecture code);
- The survival rate $\beta$ (or `survival_rate` in the lecture code).

Exploring that last value is the most important thing! See if you can find the survival rate at which the population switches from dying out to surviving. Does changing the initial `x` or `N_steps` have a similar effect?


Once you have a feel for this system, I want you to calculate the [eigenvalues](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.linalg.eig.html) of the matrix $A$ in the cell below. You can do this by using `lambda_vals,V = np.linalg.eig(A)`, and then printing `lambda_vals`. (The matrix $V$ contains the *eigenvectors* -- we don't need to worry about these for now.)


Go back and change the survival rate, and look at how the eigenvalues change. 

**What do you notice happens to the eigenvalues as the population switches from going extinct to surviving?**

This is a deeply important result. We will discuss it more in lectures this week. It turns out that the long-term behaviour of this system (and therefore the survival or extinction of the species) is 100% determined by your observation about these eigenvalues -- if you know the eigenvalues you understand everything there is to know about the system! 

That is a pretty great *summary statistic*, and piece of *dimension reduction* for this complicated system, down to effectively one number.



### Eigenfaces: eigenvectors and principal component analysis in action

Speaking of dimension reduction, let's demonstrate a "jewel in the crown" of linear algebra -- principal component analysis (PCA), which is an application of eigenvector analysis. We'll step through this using a classic example of [eigenfaces](https://sites.cs.ucsb.edu/~mturk/Papers/jcn.pdf) -- finding the significant features defining a face from images. This is the basis of modern facial recognition technology. We'll step through how the algorithm works.

First we need some images of faces, which we can obtain from the machine learning toolbox `scikit-learn`. The following code block takes a little while to download 400 images of faces from the "Olivetti" dataset.

In [ ]:
from sklearn import datasets

faces = datasets.fetch_olivetti_faces()
faces.images.shape

This is actually a stack of 400 $64 \times 64$ matrices -- we can take a look at one:

In [ ]:
B = faces.images[0]
print(B)

Each entry is a grayscale value between 0 and 1, for that pixel in the image. (Yes, an image is nothing more than a matrix! And as we're about to find out, image compression, facial recognition, etc, is nothing more than linear algebra.)

This matrix looks better plotted as an image:

In [ ]:
face_index = 0
plt.imshow(faces.images[face_index], cmap=plt.cm.bone)
plt.show()

Look at a few more images by changing `face_index` above!

The first step in doing linear algebra on this dataset is to rearrange each $64 \times 64$ image into a vector of length $64^2 = 4096$. We stack all these vectors into a new matrix $X$.

In [ ]:
n_pixels = 64
n_images = faces.images.shape[0]
X = np.zeros((n_images,n_pixels*n_pixels))
for i in range(n_images):
    X[i,:] = faces.images[i].reshape(n_pixels*n_pixels)
X.shape    

#### PCA step 1: Calculate the mean image $\bar{x}$

We calculate the mean image by averaging down each column. This also allows us to make a neat picture of the "average face", like you see going around the Internet sometimes: [here](https://www.smh.com.au/technology/technology-plots-the-average-face-of-sydney--and-the-rest-of-the-world-20110211-1ap3w.html) or [here](https://www.reddit.com/search/?q=average%20face).

Calculate the mean image using `np.mean`. You might need to look up what the optional `axis` argument does, and make sure you calculate down columns, not along rows.

In [ ]:
Xmean = 

In [ ]:
plt.imshow(Xmean.reshape(n_pixels,n_pixels),cmap=plt.cm.bone)
plt.show()

#### PCA step 2: Calculate differences $X'$ from the mean image

You should be a bit freaked out by the step below: I'm subtracting a vector from a matrix, which is not allowed in linear algebra. Like most modern programming languages, Python guesses what we actually want here, and subtracts `Xmean` from each row of `X`. And fortunately this is correct in this case! But in general, be careful.

[Question: how would you do this the correct linear algebra way, i.e., create a matrix of size $400\times4096$ containing copies of the $1\times4096$ row vector $\bar{x}$? Try it out.]

In [ ]:
Xdiff = X - Xmean
Xdiff.shape

#### PCA step 3: Calculate the covariance matrix

The covariance matrix is an important object in statistics which specifies how each pair of pixels vary with respect to each other across the dataset. If you remember what variance/standard deviation are, it's like that but for pairs of covarying datasets; if you don't, no worries.

In [ ]:
covariance_matrix = np.matmul(Xdiff.T,Xdiff)/(n_images-1)
covariance_matrix.shape

#### PCA step 4: The principal components are the eigenvectors of the covariance matrix

The principal components are vectors pointing in the directions of maximum variation of the data. PC1 corresponds to the largest eigenvalue, and describes the largest amount of variation. In the context of images, this is going to be the image which looks contains most of the information about faces -- check it out below. (Takes a few seconds; it's a big matrix!)

In [ ]:
d,V = np.linalg.eig(covariance_matrix)

In [ ]:
plt.imshow(V[:,0].reshape(n_pixels,n_pixels).real,cmap=plt.cm.bone)
plt.show()

Spooky! By contrast, the PCs corresponding to smaller eigenvalues describe less variation in the data, and so look much less face-like. Look at the 100,200,400,600th PC as an image in the cell below:

Facial recognition works by calculating these PCs/eigenfaces for a large "training" corpus, and then calculating the distance between a new image/vector, and the top PC vectors. If the distance is small, the image is probably a face; if it's large, it probably isn't!

This is just one catchy example, but PCA has countless other data science applications. We will talk more about these in lectures later in week 7 or week 8.


##### An aside: PCA and image compression

Eigenvectors/PCs are super important for image compression as well. The idea here is that it's expensive to carry around a full 4096-dimensional representation of an image, so we compress by carrying around how much of the image is captured by the first $N$ PCs instead. For example, instead of the full $400 \times 4096$ matrix $X$ we can *project* onto the first $N=100$ PCs:

In [ ]:
n_components = 100

w = np.matmul(Xdiff,V[:,:n_components])
w.shape


And then we can recover how much of the original image is captured by those first $N$ PCs as follows:

In [ ]:
T = np.matmul(V[:,:n_components],w.T).T+Xmean

plt.imshow(T[0,:].reshape(n_pixels,n_pixels).real,cmap=plt.cm.bone)
plt.show()

Write out the above expressions for $w$ and $T$ on paper using matrix notation. Why do we need to add `Xmean`?


Our final image doesn't look too bad, considering we've thrown away all but 100/4096 = 2.4% of the information! This is related to the fact that those later PCs looked noisier and noisier -- they contained less useful information. If you repeat the above using say $N=400$ components you'll see that it looks pretty close to the original image.